<div class="div_image pull-right">
    <div class = "image image_topic pull-right">
        <img src = "https://i.imgur.com/EGtMXKh.jpg?1">
    </div>
</div>

# <b>Capstone Project: Predicting Dota 2 Match Wins using Machine Learning and Recommender System (Recommender System)</b>


# Import Libraries


In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tqdm

import json

import warnings
warnings.filterwarnings('ignore')


In [2]:
# Import heroes data from json file

with open('recommender\heroes.json', 'r') as fp:
    heroes = json.load(fp)
    hero_ids = [hero['id'] for hero in heroes]


def get_hero_human_readable(hero_id):
    for hero in heroes:
        if hero['id'] == hero_id:
            return hero['localized_name']
    return 'Unknown hero: %d' % hero_id


In [3]:
# Store Variables that may change
FINAL_HERO_ID = hero_ids[-1]
NUM_FEATURES = FINAL_HERO_ID * 2
ENV_PATH = 'recommender\model.h5'
radiant_team = [45, 39, 87, 66]
dire_team = [4, 5, 13, 99, 75]

missing_ids = []
for num in range(0, FINAL_HERO_ID):
    if num not in hero_ids:
        missing_ids.append(num)


In [4]:
class NNPredictor:
    '''Load Model from the environment path for us to train'''

    def __init__(self, env_path=ENV_PATH):
        self.model = tf.keras.models.load_model(env_path)

    def transform(self, my_team, their_team):
        '''Transform our inputs into the tensorflow input array of shape (1, 246). Slice list of IDs from heroes list '''
        X = np.zeros((1, (NUM_FEATURES+1)))
        '''Slice hero_id chosen onto the array, add number of heroes to hero_id for dire team'''
        for hero_id in my_team:
            X[0][(hero_id)] = 1
        for hero_id in their_team:
            X[0][(hero_id + FINAL_HERO_ID)] = 1

        missing_ids = []
        for num in range(0, FINAL_HERO_ID):
            if num not in hero_ids:
                missing_ids.append(num)

        dire_ids = []
        for id in missing_ids:
            if id > 0:
                dire_id = id + FINAL_HERO_ID
                dire_ids.append(dire_id)

        missing_ids = missing_ids + dire_ids

        X = np.delete(X, missing_ids, axis=1)
        return X

    def recommend(self, my_team, their_team, hero_candidates):
        team_possibilities = [(candidate, my_team + [candidate])
                              for candidate in hero_candidates]

        prob_candidate_pairs = []
        for candidate, team in team_possibilities:
            query = self.transform(team, their_team)
            prob = self.score(query)
            prob_candidate_pairs.append((prob, candidate))
        prob_candidate_pairs = sorted(prob_candidate_pairs, reverse=True)[0:3]
        return prob_candidate_pairs

    def score(self, query):
        rad_prob_1 = self.model.predict(query, verbose=False)[0][0]
        return rad_prob_1

    def predict(self, dream_team, their_team):
        '''Returns the probability of the dream_team winning against their_team.'''
        dream_team_query = self.transform(dream_team, their_team)
        return self.score(dream_team_query)


In [5]:
def main():
    my_team = radiant_team
    their_team = dire_team
    print(
        f'My Team: {[get_hero_human_readable(hero_id) for hero_id in my_team]}')
    print(
        f'Their Team:{[get_hero_human_readable(hero_id) for hero_id in their_team]}')
    print('Recommend:')

    engine = Engine(NNPredictor())
    recommendations = engine.recommend(my_team, their_team)
    print(f'{[(prob, get_hero_human_readable(hero))for prob, hero in recommendations]}')


In [6]:
class Engine:

    def __init__(self, algorithm):
        self.algorithm = algorithm

    def get_candidates(self, my_team, their_team):
        '''Return a list of hero_IDs to consider for recommending'''
        ids = [
            i for i in hero_ids if i not in my_team and i not in their_team and i not in missing_ids]
        return ids

    def recommend(self, my_team, their_team, human_readable=False):
        '''Return a list of (hero, probability of winning with hero_added) recommend to complete my_team'''

        assert len(my_team) <= 5
        assert len(their_team) <= 5

        hero_candidates = self.get_candidates(my_team, their_team)
        return self.algorithm.recommend(my_team, their_team, hero_candidates)

    def predict(self, dream_team, their_team):
        '''Return the probability of the dream_team winning against their team'''
        return self.algorithm.predict(dream_team, their_team)


In [7]:
if __name__ == '__main__':
    main()


My Team: ['Pugna', 'Queen of Pain', 'Disruptor', 'Chen']
Their Team:['Bloodseeker', 'Crystal Maiden', 'Puck', 'Bristleback', 'Silencer']
Recommend:
[(0.57585835, 'Visage'), (0.5426379, 'Bounty Hunter'), (0.5331673, 'Lone Druid')]


In [8]:
# Flask implementation

import flask
from flask import Flask, render_template, session, redirect, url_for
from flask_wtf import FlaskForm
from wtforms import RadioField, SelectField, SubmitField
from wtforms.validators import DataRequired
import numpy as np
import os


In [9]:
# Creating a dictionary for choices for the form to use

heroes_json = 'recommender\heroes.json'


def choice(heroes_json):
    with open(heroes_json, 'r') as fp:
        heroes = json.load(fp)

    hero_dict = {heroes[num]['id']: heroes[num]['localized_name']
                 for num in range(0, len(heroes))}

    choices = list(hero_dict.items())
    return choices


In [10]:
# Creating a hero dictionary to map the values

def hero_dictionary(heroes_json):
    with open(heroes_json, 'r') as fp:
        heroes = json.load(fp)

    hero_dict = {heroes[num]['id']: heroes[num]['localized_name']
                 for num in range(0, len(heroes))}

    return hero_dict

In [ ]:
#Flask App

app = Flask(__name__, template_folder='html_templates')

app.config['SECRET_KEY'] = 'mysecretkey'


class HeroForm(FlaskForm):

    radiant_hero_1 = SelectField(
        u'First Radiant Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    radiant_hero_2 = SelectField(
        u'Second Radiant Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    radiant_hero_3 = SelectField(
        u'Third Radiant Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    radiant_hero_4 = SelectField(
        u'Fourth Radiant Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    radiant_hero_5 = SelectField(
        u'Fifth Radiant Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    dire_hero_1 = SelectField(u'First Dire Hero:',
                              choices=choice(heroes_json), validators=[DataRequired()])
    dire_hero_2 = SelectField(
        u'Second Dire Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    dire_hero_3 = SelectField(u'Third Dire Hero:',
                              choices=choice(heroes_json), validators=[DataRequired()])
    dire_hero_4 = SelectField(
        u'Fourth Dire Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    dire_hero_5 = SelectField(u'Fifth Dire Hero:',
                              choices=choice(heroes_json), validators=[DataRequired()])
    submit = SubmitField('Recommend')


@app.route('/', methods=['GET', 'POST'])
def home():
    form = HeroForm()
    if form.validate_on_submit():
        session['radiant_hero_1'] = form.radiant_hero_1.data
        session['radiant_hero_2'] = form.radiant_hero_2.data
        session['radiant_hero_3'] = form.radiant_hero_3.data
        session['radiant_hero_4'] = form.radiant_hero_4.data
        session['radiant_hero_5'] = form.radiant_hero_5.data
        session['dire_hero_1'] = form.dire_hero_1.data
        session['dire_hero_2'] = form.dire_hero_2.data
        session['dire_hero_3'] = form.dire_hero_3.data
        session['dire_hero_4'] = form.dire_hero_4.data
        session['dire_hero_5'] = form.dire_hero_5.data

        return redirect(url_for("recommendation"))

    return render_template('home.html', form=form)


@app.route('/recommendation')
def recommendation():

    radiant_team = (filter(lambda x: x != 'Nil', [session['radiant_hero_1'],
                                                  session['radiant_hero_2'],
                                                  session['radiant_hero_3'],
                                                  session['radiant_hero_4'],
                                                  session['radiant_hero_5']]))
    dire_team = (filter(lambda x: x != 'Nil', [session['dire_hero_1'],
                                               session['dire_hero_2'],
                                               session['dire_hero_3'],
                                               session['dire_hero_4'],
                                               session['dire_hero_5']]))
    radiant_team = [[int(s) for s in sublist] for sublist in radiant_team]
    dire_team = [[int(s) for s in sublist] for sublist in dire_team]

    my_team = radiant_team
    their_team = dire_team

    if len(my_team) >= 5:
        return 'Your Team is Full! Please remove a hero from your team for the system to recommend.'
    elif len(my_team) == 0 and len(my_team) == 0:
        return 'Your Team is Empty! Please fill in at least one hero to continue.'
    else:
        engine = Engine(NNPredictor())
        prob_recommendation_pairs = engine.recommend(my_team, their_team)
        recommendations = [hero for prob,
                           hero in prob_recommendation_pairs]
        heroes = [(hero_dictionary(heroes_json)[hero])
                  for hero in recommendations]
        prob = (round(engine.predict(my_team, their_team) * 100, 2))
        probability = f"Probability of your team winning is {prob}%"
        return render_template('recommendation.html', prediction_text='{}'.format((heroes, probability)))


if __name__ == "__main__":
    app.debug = True
    app.run()
